In [1]:
# model使用二次方程

import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

import matplotlib.pyplot as plt

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0] # y
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # x
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [3]:
# y = ax*x + bx + c
def model(input_tensor:torch.tensor,a,b,c):
    return input_tensor * input_tensor * a + b * input_tensor + c

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [6]:
from torch import optim

params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

In [7]:
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        
        # optimizer.zero_grad()的作用是将所有参数的梯度值置零，即将每个参数的梯度张量中的所有元素设置为0。
        # 这个操作会遍历优化器中的所有参数，并对其梯度进行归零
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [8]:
training_loop(
    n_epochs = 2000, 
    optimizer = optimizer,
    params = params,
    t_u = t_u, 
    t_c = t_c)

Epoch 500, Loss 6.108696
Epoch 1000, Loss 5.984358
Epoch 1500, Loss 5.809033
Epoch 2000, Loss 5.586317


tensor([ 0.0044, -0.0094, -1.0349], requires_grad=True)

In [9]:
# 使用训练集和测试集

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 9,  2,  0,  5,  8,  7,  3,  6, 10]), tensor([4, 1]))

In [10]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [11]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params) # <1>
        train_loss = loss_fn(train_t_p, train_t_c)
                             
        val_t_p = model(val_t_u, *params) # <1>
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
            
    return params

In [12]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un, # <1> 
    val_t_u = val_t_un, # <1> 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 691.0059, Validation loss 607.3423
Epoch 2, Training loss 360941.6562, Validation loss 305255.9375
Epoch 3, Training loss 192995392.0000, Validation loss 163589664.0000
Epoch 500, Training loss nan, Validation loss nan
Epoch 1000, Training loss nan, Validation loss nan
Epoch 1500, Training loss nan, Validation loss nan
Epoch 2000, Training loss nan, Validation loss nan
Epoch 2500, Training loss nan, Validation loss nan
Epoch 3000, Training loss nan, Validation loss nan


tensor([nan, nan, nan], requires_grad=True)

In [14]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un, # <1> 
    val_t_u = val_t_un, # <1> 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 691.0059, Validation loss 607.3423
Epoch 2, Training loss 670.0638, Validation loss 588.7557
Epoch 3, Training loss 649.4578, Validation loss 570.4668
Epoch 500, Training loss 6.4419, Validation loss 2.6582
Epoch 1000, Training loss 3.7027, Validation loss 3.2041
Epoch 1500, Training loss 2.8668, Validation loss 3.7664
Epoch 2000, Training loss 2.7170, Validation loss 4.0271
Epoch 2500, Training loss 2.6631, Validation loss 4.0576
Epoch 3000, Training loss 2.6055, Validation loss 4.0016


tensor([ 0.5365, -0.2766, -4.0127], requires_grad=True)